<a href="https://colab.research.google.com/github/AdamClarkStandke/Emerging-Neural-Network-Models/blob/main/SiameseNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meta Learning: Siamese Networks

Siamese Networks are a form of Meta Learning and is considered to be an emerging neural netwok design, similar to transfer leaning but differnt. As the author of the book titled [Advanced Deep Learning with Python](https://www.amazon.com/Advanced-Deep-Learning-Python-next-generation/dp/178995617X) details:

> Meta Learning, also referred to as learning to learn, allows machine learning (ML) algorithms to leverage and channel knowledge, gained over multiple training tasks, to improve its training efficiency over a new task...[this form of learning] has the ability to train with fewer samples...[allowing] for reduced training time and good perfromance when there is not enough traning data.

Siamese networks were proposed in the paper [Siamese Neural Netowrks for One shot Image Reconation](https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf). As the authors of the paper detail:

> In this paper, we explore a method for learning siamese neural networks which employ a unique structure to naturally rank similarity between inputs. Once a network has been tuned, we can then capitalize on powerful discriminative features to generalize the predictive power of the network not just to new data, but to entirely new classes from unknown distributions.



---








## Creating equal number of true/false pairs of samples and Loading the MNIST Data

I will be implementing the Siamese Network as detailed in the book [Advanced Deep Learning with Python](https://www.amazon.com/Advanced-Deep-Learning-Python-next-generation/dp/178995617X). Before doing so, a helper function called ```create_pairs``` is created. Since the book trains and tests using the MNIST image dataset, each dataset sample consists of an input pair of two MNIST images and a binary label, which indicates whether they are from the same class





In [1]:
# importing necessary packages
import random
import numpy as np
import tensorflow as tf

In [3]:
# creating the training/testing dataset
def create_pairs(inputs: np.ndarray, labels: np.ndarray):
  num_classes= 10
  digit_indices = [np.where(labels==i)[0] for i in range(num_classes)]
  pairs = list()
  labels = list()
  n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
  for d in range(num_classes):
    for i in range(n):
      z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
      pairs += [[inputs[z1], inputs[z2]]]
      inc = random.randrange(1, num_classes)
      dn = (d + inc) % num_classes
      z1, z2 = digit_indices[d][i], digit_indices[dn][i]
      pairs += [[inputs[z1], inputs[z2]]]
      labels += [1, 0]
  return np.array(pairs), np.array(labels, dtype=np.float32)

In [5]:
# Load the train and test MNIST datasets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_train /= 255
x_test /= 255
input_shape = x_train.shape[1:]

# Create true/false training and testing pairs
train_pairs, tr_labels = create_pairs(x_train, y_train)
test_pairs, test_labels = create_pairs(x_test, y_test)

11490434/11490434 [==============================] - 2s 0us/step


## Creating the Base Network of the Siamese System

This portion of code will implement the base network of the Siamese System, in which two idential base networks (with shared parameters) are used to output embedding vectors.

In [4]:
def create_base_network():
    """The shared encoding part of the siamese network"""

    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation='relu'),
    ])

## Building the Siamese System

This code portion will build the Siamese System, which includes the base network, the 2 siamese paths of ```encoder_a ```,and ```encoder_b```, the ```l1_dist measure``` and the combined ```model```. In other words implementing the following formula: $L1=|f_{\theta}(x_i)-f_{\theta}(x_j)|$ where $f_{\theta}$ are ```encoder_a``` and ```encoder_b```

In [6]:
# Create the siamese network
# Start from the shared layers
base_network = create_base_network()

# Create first half of the siamese system
input_a = tf.keras.layers.Input(shape=input_shape)

# Note how we reuse the base_network in both halfs
encoder_a = base_network(input_a)

# Create the second half of the siamese system
input_b = tf.keras.layers.Input(shape=input_shape)
encoder_b = base_network(input_b)

# Create the the distance measure
l1_dist = tf.keras.layers.Lambda(lambda embeddings: tf.keras.backend.abs(embeddings[0] - embeddings[1]))\
 ([encoder_a, encoder_b])

# Final fc layer with a single logistic output for the binary classification
flattened_weighted_distance = tf.keras.layers.Dense(1, activation='sigmoid') \
    (l1_dist)

# Build the model
model = tf.keras.models.Model([input_a, input_b], flattened_weighted_distance)

### Training the Validating the model

In [ ]:
# Train
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit([train_pairs[:, 0], train_pairs[:, 1]], tr_labels,
          batch_size=128,
          epochs=20,
          validation_data=([test_pairs[:, 0], test_pairs[:, 1]], test_labels))